In [11]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

# Make sure that caffe is on the python path:
caffe_root = '/usr/local/google/home/zyan/proj/caffe_private_recurrent_2d/'  # this file is expected to be in {caffe_root}/examples
local_caffe_root='/usr/local/google/home/zyan/proj/caffe/'
import sys
sys.path.insert(0, caffe_root + 'python')

import caffe

# Set the right path to your model definition file, pretrained model weights,
# and the image you would like to classify.
model_name = 'grid_1_lay_HP_nolbw_nopeep_seg_lr5e-2'
iteration = 18000
MODEL_FILE = local_caffe_root+\
'google_extra/examples/mnist_segmentation/grid_1_lay_HP_nolbw_nopeep_seg_deploy.prototxt'
PRETRAINED = local_caffe_root+\
'google_extra/examples/mnist_segmentation/grid_lstm_3d_1_lay/%s_iter_%d.caffemodel'%(model_name, iteration)

import os
from struct import *
import matplotlib.pyplot as plt

plt.rcParams['figure.figsize'] = (10, 10)
plt.rcParams['image.interpolation'] = 'nea/usr/local/google/home/zyan/proj/cafferest'
# plt.rcParams['image.cmap'] = 'gray'

caffe.set_mode_cpu()
net = caffe.Classifier(MODEL_FILE, PRETRAINED, caffe.TEST)


In [12]:
[ (layer, net.blobs[layer].data.shape) for layer in net.blobs]

[('data', (100, 1, 28, 28)),
 ('data_input_0_split_0', (100, 1, 28, 28)),
 ('data_input_0_split_1', (100, 1, 28, 28)),
 ('conv1_state', (100, 32, 28, 28)),
 ('conv1_state_dimension_transpose', (28, 28, 100, 32)),
 ('conv1_hidden', (100, 32, 28, 28)),
 ('conv1_hidden_dimension_transpose', (28, 28, 100, 32)),
 ('grid_lstm_3d_1_posy_posx_state', (28, 28, 100, 32)),
 ('grid_lstm_3d_1_posy_posx_hidden', (28, 28, 100, 32)),
 ('grid_lstm_3d_1_posy_negx_state', (28, 28, 100, 32)),
 ('grid_lstm_3d_1_posy_negx_hidden', (28, 28, 100, 32)),
 ('grid_lstm_3d_1_negy_posx_state', (28, 28, 100, 32)),
 ('grid_lstm_3d_1_negy_posx_hidden', (28, 28, 100, 32)),
 ('grid_lstm_3d_1_negy_negx_state', (28, 28, 100, 32)),
 ('grid_lstm_3d_1_negy_negx_hidden', (28, 28, 100, 32)),
 ('grid_lstm_3d_1_negy_negx_state_dimension_transpose', (100, 32, 28, 28)),
 ('grid_lstm_3d_1_negy_negx_hidden_dimension_transpose', (100, 32, 28, 28)),
 ('concat_state_hidden', (100, 64, 28, 28)),
 ('conv1', (100, 11, 28, 28)),
 ('prob', 

In [3]:
test_img_data_file='/usr/local/google/home/zyan/data/mnist/t10k-images.idx3-ubyte'  
with open(test_img_data_file,'rb') as f:
    magic_num_str = f.read(4)
    magic_num = (unpack('>i',magic_num_str))[0]
    num_img_str = f.read(4)
    num_img = (unpack('>i',num_img_str))[0]
    num_row_str = f.read(4)
    num_row = (unpack('>i',num_row_str))[0]
    num_col_str = f.read(4)
    num_col = (unpack('>i',num_col_str))[0]
    pix_byte = f.read(num_img*num_row*num_col)
print 'num_img %d' % num_img
print 'num_row %d num_col %d' % (num_row, num_col)

num_img 10000
num_row 28 num_col 28


In [32]:
test_img_label_file = '/usr/local/google/home/zyan/data/mnist/t10k-labels.idx1-ubyte'
with open(test_img_label_file,'rb') as f:
    magic_num_str = f.read(4)
    magic_num = (unpack('>i',magic_num_str))[0]
    num_img_str = f.read(4)
    num_img = (unpack('>i',num_img_str))[0]
    print 'num_img %d' % num_img
    label_byte = f.read(num_img)
    test_gt_labels = np.zeros((num_img),dtype=np.int32)
    for i in range(num_img):
        test_gt_labels[i] = (unpack('>B',label_byte[i]))[0]

num_img 10000


In [4]:
img_data=np.zeros((num_img, num_row, num_col, 3), dtype=np.uint8)
net_in_data=np.zeros((num_img, 1, num_row, num_col), dtype=np.single)
pix_c=0
for n in range(num_img):
    for h in range(num_row):
        for w in range(num_col):
            pix_val=(unpack('>B',pix_byte[pix_c]))[0]
            img_data[n,h,w,:]=pix_val
            net_in_data[n,0,h,w]=pix_val
            pix_c+=1

In [5]:
'''transform the net input data'''
data_mean = 128
scale = 0.0078125
net_in_data = (net_in_data - data_mean) * scale

In [6]:
img_h,img_w=net.blobs['data'].data.shape[2],net.blobs['data'].data.shape[3]
batch_size = net.blobs['data'].data.shape[0]
label_num = net.blobs['prob'].data.shape[1]

pred_label = np.zeros((num_img,img_h,img_w))

print batch_size
for batch in range(num_img/batch_size):
    if batch % 10 == 0:
        print 'batch %d out of %d' % (batch, num_img/batch_size)
    start,end=batch*batch_size,(batch+1)*batch_size
    net.blobs['data'].data[...] = net_in_data[start:end,:,:,:]
    out = net.forward()
    pred_label[start:end,:,:]=np.argmax(out['prob'],axis=1)

100
batch 0 out of 100
batch 10 out of 100
batch 20 out of 100
batch 30 out of 100
batch 40 out of 100
batch 50 out of 100
batch 60 out of 100
batch 70 out of 100
batch 80 out of 100
batch 90 out of 100


In [36]:
'''classify the entire image using majority pixel voting'''
image_classify_correct = 0
for i in range(num_img):
    hist,bin_edges=np.histogram(pred_label[i,:,:].flat, bins=np.arange(12))
    if np.argmax(hist[:10]) == test_gt_labels[i]:
        image_classify_correct += 1.0

error_rate = (1.0 - (image_classify_correct / num_img)) * 100.0
print 'digit classification error : %4.2f  ' % error_rate

digit classification error : 1.25  


In [13]:
'''visualize the segmentation results'''
save_dir = local_caffe_root+'google_extra/examples/mnist_segmentation/grid_lstm_3d_1_lay/'+model_name+'_results/'
if not os.path.exists(save_dir):
    os.mkdir(save_dir)
    
for batch in range(num_img/50):
# for batch in range(1):
    count = 0
    fig, axes = plt.subplots(nrows=10, ncols=10)
    
    for ax in axes.flat:
        img_id = batch * 50 + count/2
        if count % 2== 0:
            img_data_for_show = img_data[img_id, :, :, 0]
            im = ax.imshow(img_data_for_show,cmap='gray', interpolation='nearest')
        else:
            loc_pred_label = np.copy(pred_label[img_id,:,:])
            '''assign a corner pixel with label 0 so that the colormap can
            have full label range [0,10]'''
            loc_pred_label[0,0]=0
            im = ax.imshow(loc_pred_label, cmap='jet',interpolation='nearest')
        ax.set_axis_off()
        count += 1
    
    fig.subplots_adjust(right=0.85)
    cbar_ax = fig.add_axes([0.90, 0.15, 0.03, 0.7])
    cbar=fig.colorbar(im,cax=cbar_ax,ticks=np.arange(label_num))
    for t in cbar.ax.get_yticklabels():
        t.set_fontsize(15)   
        
        
    fig.suptitle('batch %d'%(batch),fontsize=15)
            
    plt.savefig(save_dir+'batch_%d.jpg'%batch)
    plt.close()    

In [ ]:
'''study a single test case'''
img_id = 67
plt.imshow(img_data[img_id,:,:,:])

In [ ]:
net.blobs['data'].data[0,:,:,:] = net_in_data[img_id,:,:,:]
out = net.forward()
pred_prob=out['prob'][0,:,:,:]
pred_label=np.argmax(out['prob'][0,:,:,:],axis=0)
plt.imshow(pred_label, cmap='jet')
plt.axis('off')
cbar=plt.colorbar(ticks=np.arange(label_num))